## Load Data

In [18]:
import os
import sys
import pandas as pd
import logging
from google.cloud import bigquery
from hashlib import md5
from typing import List
import json

# **** SETUP ****

# change to match your filesystem
DATA_DIR = "./data/air_travel/"
DEFAULT_RECEIPTS_FILE = os.path.join(DATA_DIR, "tickets.json")
PROJECT_NAME = "deb-dev-dw"
DATASET_NAME = "air_travel"

data = []
with open('./data/air_travel/tickets.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))

df = pd.DataFrame.from_dict(pd.json_normalize(data), orient='columns')

display(df.head(n=10))

eticket_num confirmation ticket_date   price seat  status  \
0  498-938211-0795       ZVFDC4  2022-03-23  723.42  31I  active   
1  482-850738-6048       IL5GUI  2022-03-23  765.18  29B  active   
2  275-207321-8092       CYEFBC  2022-03-21  753.89  26I  active   
3  246-793315-3102       ZNGPC2  2022-03-22  793.89  15A  active   
4  091-128904-1226       MGSBD9  2022-03-24  820.25  17F  active   
5  115-196069-8963       XFYQC0  2022-03-23  892.69  18C  active   
6  396-673460-1326       N5UOOZ  2022-03-23  889.53   3C  active   
7  380-894599-8109       PAA19Y  2022-03-22  706.78   7D  active   
8  614-960971-2686       EF4BHJ  2022-03-23  486.40  24J  active   
9  481-321233-0702       FVM9EE  2022-03-23  855.93  16A  active   

             airline.name airline.iata airline.icao  airline.callsign  ...  \
0  China Eastern Airlines           MU          CES     CHINA EASTERN  ...   
1       Hawaiian Airlines           HA          HAL          HAWAIIAN  ...   
2                Wizz Air           W6          WZZ          WIZZ AIR  ...   
3                 AirAsia           AK          AXM     ASIAN EXPRESS  ...   
4         Xiamen Airlines           MF          CXA        XIAMEN AIR  ...   
5         Air New Zealand           NZ          ANZ       NEW ZEALAND  ...   
6                Jeju Air           7C          JJA          JEJU AIR  ...   
7       American Airlines           AA          AAL          AMERICAN  ...   
8        Juneyao Airlines           HO          DKH  JUNEYAO AIRLINES  ...   
9         Royal Air Maroc           AT          RAM    ROYALAIR MAROC  ...   

  passenger.last_name passenger.gender passenger.birth_date  \
0               Brown                M           1969-02-17   
1                Kent                F           1998-08-05   
2              Tucker                F           1965-01-22   
3               Yates               NB           1975-03-31   
4          Villanueva               NB           1945-08-14   
5                Hall               NB           1944-08-31   
6            Thompson                M           1968-05-02   
7              Garcia                F           1950-02-12   
8               Clark                F           1991-11-09   
9                Cook                M           1976-07-29   

                   passenger.email                   passenger.street  \
0      robert.brown.69@hotmail.com                    5007 Thomas Way   
1        laura.kent.98@hotmail.com                13991 Davis Village   
2       lisa.tucker.65@hotmail.com                   04135 Marvin Via   
3       matthew.yates.75@yahoo.com      76045 Samantha Road Suite 111   
4  megan.villanueva.45@hotmail.com      848 Melissa Springs Suite 947   
5          sarah.hall.44@gmail.com  75420 Michael Mountains Suite 485   
6       seth.thompson.68@yahoo.com    22455 Higgins Junction Apt. 042   
7     jennifer.garcia.50@gmail.com                  6607 Sharp Common   
8         becky.clark.91@gmail.com                   691 Jones Cliffs   
9       ronald.cook.76@hotmail.com                 93328 Davis Island   

           passenger.city  passenger.state  passenger.zip  origin destination  
0          Lake Hollystad               DC          20027     NaN         NaN  
1  North Catherineborough               PA          16516     NaN         NaN  
2        North Kristabury               MA          01093     NaN         NaN  
3            Lake Jeffrey               DE          19898     NaN         NaN  
4              Kellerstad               TX          76177     NaN         NaN  
5            New Victoria               HI          96727     NaN         NaN  
6               New Keith               OR          97405     NaN         NaN  
7                Chadstad               VA          22121     NaN         NaN  
8            Michaelburgh               TX          76003     NaN         NaN  
9           Rodriguezside               MD          21408     NaN         NaN  

[10 rows x 40 column

In [19]:
# modify column name to work better with big query
df.columns = df.columns.str.replace(".", "_")
display(df.head(n=10))

/Users/drewwhite/Desktop/Epicodus/data_week_8/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


eticket_num confirmation ticket_date   price seat  status  \
0  498-938211-0795       ZVFDC4  2022-03-23  723.42  31I  active   
1  482-850738-6048       IL5GUI  2022-03-23  765.18  29B  active   
2  275-207321-8092       CYEFBC  2022-03-21  753.89  26I  active   
3  246-793315-3102       ZNGPC2  2022-03-22  793.89  15A  active   
4  091-128904-1226       MGSBD9  2022-03-24  820.25  17F  active   
5  115-196069-8963       XFYQC0  2022-03-23  892.69  18C  active   
6  396-673460-1326       N5UOOZ  2022-03-23  889.53   3C  active   
7  380-894599-8109       PAA19Y  2022-03-22  706.78   7D  active   
8  614-960971-2686       EF4BHJ  2022-03-23  486.40  24J  active   
9  481-321233-0702       FVM9EE  2022-03-23  855.93  16A  active   

             airline_name airline_iata airline_icao  airline_callsign  ...  \
0  China Eastern Airlines           MU          CES     CHINA EASTERN  ...   
1       Hawaiian Airlines           HA          HAL          HAWAIIAN  ...   
2                Wizz Air           W6          WZZ          WIZZ AIR  ...   
3                 AirAsia           AK          AXM     ASIAN EXPRESS  ...   
4         Xiamen Airlines           MF          CXA        XIAMEN AIR  ...   
5         Air New Zealand           NZ          ANZ       NEW ZEALAND  ...   
6                Jeju Air           7C          JJA          JEJU AIR  ...   
7       American Airlines           AA          AAL          AMERICAN  ...   
8        Juneyao Airlines           HO          DKH  JUNEYAO AIRLINES  ...   
9         Royal Air Maroc           AT          RAM    ROYALAIR MAROC  ...   

  passenger_last_name passenger_gender passenger_birth_date  \
0               Brown                M           1969-02-17   
1                Kent                F           1998-08-05   
2              Tucker                F           1965-01-22   
3               Yates               NB           1975-03-31   
4          Villanueva               NB           1945-08-14   
5                Hall               NB           1944-08-31   
6            Thompson                M           1968-05-02   
7              Garcia                F           1950-02-12   
8               Clark                F           1991-11-09   
9                Cook                M           1976-07-29   

                   passenger_email                   passenger_street  \
0      robert.brown.69@hotmail.com                    5007 Thomas Way   
1        laura.kent.98@hotmail.com                13991 Davis Village   
2       lisa.tucker.65@hotmail.com                   04135 Marvin Via   
3       matthew.yates.75@yahoo.com      76045 Samantha Road Suite 111   
4  megan.villanueva.45@hotmail.com      848 Melissa Springs Suite 947   
5          sarah.hall.44@gmail.com  75420 Michael Mountains Suite 485   
6       seth.thompson.68@yahoo.com    22455 Higgins Junction Apt. 042   
7     jennifer.garcia.50@gmail.com                  6607 Sharp Common   
8         becky.clark.91@gmail.com                   691 Jones Cliffs   
9       ronald.cook.76@hotmail.com                 93328 Davis Island   

           passenger_city  passenger_state  passenger_zip  origin destination  
0          Lake Hollystad               DC          20027     NaN         NaN  
1  North Catherineborough               PA          16516     NaN         NaN  
2        North Kristabury               MA          01093     NaN         NaN  
3            Lake Jeffrey               DE          19898     NaN         NaN  
4              Kellerstad               TX          76177     NaN         NaN  
5            New Victoria               HI          96727     NaN         NaN  
6               New Keith               OR          97405     NaN         NaN  
7                Chadstad               VA          22121     NaN         NaN  
8            Michaelburgh               TX          76003     NaN         NaN  
9           Rodriguezside               MD          21408     NaN         NaN  

[10 rows x 40 column

## Define table schema for dimension tables

In [20]:
# setting table schema for dims tables

DIMS_TABLE_METADATA = {  
  # for airlines
      'airlines': {
        'table_name': 'airlines',
        'schema': [
            bigquery.SchemaField('airline_iata', 'STRING', mode='REQUIRED'),
            bigquery.SchemaField('airline_name', 'STRING', mode='REQUIRED'),
            bigquery.SchemaField('airline_icao', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('airline_callsign', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('airline_country', 'STRING', mode='NULLABLE'),
        ],
    },
    # for airports
      'airports': {
        'table_name': 'airports',
        'schema': [
            bigquery.SchemaField('airport_iata', 'STRING', mode='REQUIRED'),
            bigquery.SchemaField('airport_name', 'STRING', mode='REQUIRED'),
            bigquery.SchemaField('airport_city', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('airport_icao', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('airport_latitude', 'FLOAT', mode='NULLABLE'),
            bigquery.SchemaField('airport_longitude', 'FLOAT', mode='NULLABLE'),
            bigquery.SchemaField('airport_altitude', 'FLOAT', mode='NULLABLE'),
            bigquery.SchemaField('airport_tz_timezone', 'FLOAT', mode='NULLABLE'),
        ],
    },
    # for passengers
      'passengers': {
        'table_name': 'passengers',
        'schema': [
            bigquery.SchemaField('passenger_id', 'STRING', mode='REQUIRED'),
            bigquery.SchemaField('passenger_email', 'STRING', mode='REQUIRED'),
            bigquery.SchemaField('passenger_first_name', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('passenger_last_name', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('passenger_birth_date', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('passenger_street', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('passenger_city', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('passenger_state', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('passenger_zip', 'INTEGER', mode='NULLABLE'),
            bigquery.SchemaField('effective_start_date', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('effective_end_date', 'STRING', mode='NULLABLE'),
        ],
    },
}

# **** SETUP LOGGING ****
# setup logging and logger
logging.basicConfig(            # setting up the root logger
    format='[%(levelname)-5s][%(asctime)s][%(module)s:%(lineno)04d] : %(message)s',
    level=logging.INFO,
    stream=sys.stdout
)
logger: logging.Logger = logging.getLogger('root')      # alias the root logger as `logger`
logger.setLevel(logging.DEBUG)                          # programmatically reassign the logging level


# **** BIGQUERY CLIENT ****
logger.debug(f"Creating bigquery client")
client = bigquery.Client()

logger.info(f"Setup Completed")

[DEBUG][2023-01-06 12:17:45,557][2447715880:0060] : Creating bigquery client
[INFO ][2023-01-06 12:17:45,561][2447715880:0063] : Setup Completed


## Create dataset in bigquery

In [21]:

dataset_id = f"{PROJECT_NAME}.{DATASET_NAME}"
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"
dataset = client.create_dataset(dataset, exists_ok=True)
# tells me that dataset was created with name
logger.info(f"Created air_travel dataset: {dataset.full_dataset_id}")

[INFO ][2023-01-06 12:17:51,714][1320071499:0006] : Created air_travel dataset: deb-dev-dw:air_travel


In [22]:
# **** CREATE TABLES ****
logger.debug(f"Creating tables:")

for table_id in DIMS_TABLE_METADATA:
  full_table_id = f"{PROJECT_NAME}.{DATASET_NAME}.{table_id}"
  schema = DIMS_TABLE_METADATA[f'{table_id}']['schema']
  table = bigquery.Table(full_table_id, schema = schema)
  client.create_table(table)
  logger.info(f"Created table: {full_table_id}")
  # List table Schema
  table_ref = client.get_table(table)
  for column in table_ref.schema:
    print(f"\t{column.name}\t{column.field_type}") 
  print("\n")

[DEBUG][2023-01-06 12:17:55,601][3118785056:0002] : Creating tables:
[INFO ][2023-01-06 12:17:55,939][3118785056:0009] : Created table: deb-dev-dw.air_travel.airlines
	airline_iata	STRING
	airline_name	STRING
	airline_icao	STRING
	airline_callsign	STRING
	airline_country	STRING


[INFO ][2023-01-06 12:17:56,525][3118785056:0009] : Created table: deb-dev-dw.air_travel.airports
	airport_iata	STRING
	airport_name	STRING
	airport_city	STRING
	airport_icao	STRING
	airport_latitude	FLOAT
	airport_longitude	FLOAT
	airport_altitude	FLOAT
	airport_tz_timezone	FLOAT


[INFO ][2023-01-06 12:17:57,034][3118785056:0009] : Created table: deb-dev-dw.air_travel.passengers
	passenger_id	STRING
	passenger_email	STRING
	passenger_first_name	STRING
	passenger_last_name	STRING
	passenger_birth_date	STRING
	passenger_street	STRING
	passenger_city	STRING
	passenger_state	STRING
	passenger_zip	INTEGER
	effective_start_date	STRING
	effective_end_date	STRING




## Load original data frame into separate data frames to make manipulations for dimensions

In [23]:
# airlines dimension table
airlines_df = df
# columns to keep for dimension
airline_expected_cols = ['airline_iata', 'airline_name', 'airline_icao', 'airline_callsign', 'airline_country']
# groupby above columns
airlines_df = airlines_df.groupby(airline_expected_cols).all()
airlines_df = airlines_df.reset_index().loc[:, airline_expected_cols]

logger.info(f"airlines - found {len(airlines_df.index)} rows")

display(airlines_df)


[INFO ][2023-01-06 12:18:08,962][1400822048:0009] : airlines - found 48 rows


airline_iata             airline_name airline_icao   airline_callsign  \
0            3U         Sichuan Airlines          CSC           SI CHUAN   
1            7C                 Jeju Air          JJA           JEJU AIR   
2            9K                 Cape Air          KAP               CAIR   
3            9S          Spring Airlines          CQH         AIR SPRING   
4            AA        American Airlines          AAL           AMERICAN   
5            AC               Air Canada          ACA         AIR CANADA   
6            AF               Air France          AFR           AIRFRANS   
7            AK                  AirAsia          AXM      ASIAN EXPRESS   
8            AS          Alaska Airlines          ASA               Inc.   
9            AT          Royal Air Maroc          RAM     ROYALAIR MAROC   
10           B6          JetBlue Airways          JBU            JETBLUE   
11           BA          British Airways          BAW          SPEEDBIRD   
12           BC         Skymark Airlines          SKY            SKYMARK   
13           CA                Air China          CCA          AIR CHINA   
14           CZ  China Southern Airlines          CSN     CHINA SOUTHERN   
15           DD                  Nok Air          NOK            NOK AIR   
16           DL          Delta Air Lines          DAL              DELTA   
17           ET       Ethiopian Airlines          ETH          ETHIOPIAN   
18           F9        Frontier Airlines          FFT    FRONTIER FLIGHT   
19           FD             Thai AirAsia          AIQ          THAI ASIA   
20           FM        Shanghai Airlines          CSH       SHANGHAI AIR   
21           G4            Allegiant Air          AAY          ALLEGIANT   
22           HA        Hawaiian Airlines          HAL           HAWAIIAN   
23           HO         Juneyao Airlines          DKH   JUNEYAO AIRLINES   
24           HR    China United Airlines          CUA           LIANHANG   
25           HU          Hainan Airlines          CHH             HAINAN   
26           IW                Wings Air          WON        WINGS ABADI   
27           MF          Xiamen Airlines          CXA         XIAMEN AIR   
28           MH        Malaysia Airlines          MAS          MALAYSIAN   
29           MU   China Eastern Airlines          CES      CHINA EASTERN   
30           MW          Maya Island Air          MYD             MYLAND   
31           NK          Spirit Airlines          NKS       SPIRIT WINGS   
32           NZ          Air New Zealand          ANZ        NEW ZEALAND   
33           PM               Tropic Air          TOS           TROPISER   
34           PR      Philippine Airlines          PAL         PHILIPPINE   
35           QR            Qatar Airways          QTR             QATARI   
36           QZ        Indonesia AirAsia          AWQ          WAGON AIR   
37           S7              S7 Airlines          SBI  SIBERIAN AIRLINES   
38           SC        Shandong Airlines          CDG           SHANDONG   
39           SJ            Sriwijaya Air          SJY          SRIWIJAYA   
40           TK         Turkish Airlines          THY            TURKAIR   
41           U6            Ural Airlines          SVR     SVERDLOVSK AIR   
42           UA          United Airlines          UAL             UNITED   
43           VN         Vietnam Airlines          HVN  VIET NAM AIRLINES   
44           W6                 Wizz Air          WZZ           WIZZ AIR   
45           WN       Southwest Airlines          SWA          SOUTHWEST   
46           WY                 Oman Air          OMA           OMAN AIR   
47           ZH        Shenzhen Airlines          CSZ       SHENZHEN AIR   

      airline_country  
0               China  
1   Republic of Korea  
2       United States  
3               China  
4       United States  
5              Canada  
6              France  
7            Malaysia  
8              ALASKA  
9             Morocco  
10      United

In [26]:
# airports dimension table

origin_df = df
dest_df = df

airport_expected_cols = ['name', 'city','country', 'iata', 'icao','latitude','longitude','altitude', 'tz_timezone']

origin_col = ['origin_' + col for col in airport_expected_cols]
dest_col = ['destination_' + col for col in airport_expected_cols]
airport_col = ['airport_' + col for col in airport_expected_cols]

# group unique origin airports
origin_df = origin_df.groupby(origin_col).all()
origin_df = origin_df.reset_index().loc[:, origin_col]

# rename columns

origin_df = origin_df.rename(columns={
    'origin_name': 'airport_name', 
    'origin_city':'airport_city', 
    'origin_country':'airport_country',
    'origin_iata':'airport_iata',
    'origin_icao':'airport_icao',
    'origin_latitude':'airport_latitude',
    'origin_longitude':'airport_longitude',
    'origin_altitude':'airport_altitude',
    'origin_tz_timezone':'airport_tz_timezone',
    })

# group unique destination airports
dest_df = dest_df.groupby(dest_col).all()
dest_df = dest_df.reset_index().loc[:, dest_col]

# rename columns

dest_df = dest_df.rename(columns={
    'destination_name': 'airport_name', 
    'destination_city':'airport_city', 
    'destination_country':'airport_country',
    'destination_iata':'airport_iata',
    'destination_icao':'airport_icao',
    'destination_latitude':'airport_latitude',
    'destination_longitude':'airport_longitude',
    'destination_altitude':'airport_altitude',
    'destination_tz_timezone':'airport_tz_timezone',
    })

# combine origin and destination
concat_df = [origin_df, dest_df]

airports_df = pd.concat(concat_df)

# group unique combined airports data

airports_df = airports_df.groupby(airport_col).all()
airports_df = airports_df.reset_index().loc[:, airport_col]

logger.info(f"airports - found {len(airports_df.index)} rows")

display(airports_df)


[INFO ][2023-01-06 12:41:37,998][4159366420:0058] : airports - found 386 rows


airport_name   airport_city  \
0         Abu Dhabi International Airport      Abu Dhabi   
1    Adolfo Suárez Madrid–Barajas Airport         Madrid   
2                     Afonso Pena Airport       Curitiba   
3              Alberto Alcolumbre Airport         Macapa   
4       Albuquerque International Sunport    Albuquerque   
..                                    ...            ...   
381                 Zhuhai Jinwan Airport         Zhuhai   
382            Zumbi dos Palmares Airport         Maceio   
383                        Zürich Airport         Zurich   
384  Ástor Piazzola International Airport  Mar Del Plata   
385   Ürümqi Diwopu International Airport         Urumqi   

          airport_country airport_iata airport_icao  airport_latitude  \
0    United Arab Emirates          AUH         OMAA             24.43   
1                   Spain          MAD         LEMD             40.47   
2                  Brazil          CWB         SBCT            -25.53   
3                  Brazil          MCP         SBMQ              0.05   
4           United States          ABQ         KABQ             35.04   
..                    ...          ...          ...               ...   
381                 China          ZUH         ZGSD             22.01   
382                Brazil          MCZ         SBMO             -9.51   
383           Switzerland          ZRH         LSZH             47.46   
384             Argentina          MDQ         SAZM            -37.93   
385                 China          URC         ZWWW             43.91   

     airport_longitude  airport_altitude   airport_tz_timezone  
0                54.65              88.0            Asia/Dubai  
1                -3.56            1998.0         Europe/Madrid  
2               -49.18            2988.0     America/Sao_Paulo  
3               -51.07              56.0     America/Fortaleza  
4              -106.61            5355.0        America/Denver  
..                 ...               ...                   ...  
381             113.38              23.0         Asia/Shanghai  
382             -35.79             387.0     America/Fortaleza  
383               8.55            1416.0         Europe/Zurich  
384             -57.57              72.0  America/Buenos_Aires  
385              87.47            2125.0         Asia/Shanghai  

[386 rows x 9 columns]

In [28]:
import uuid

pass_df = df

pass_expected_cols = ['email','first_name','last_name', 'birth_date', 'street','city','state','zip']
pass_col = ['passenger_' + col for col in pass_expected_cols]

# group unique passengers
pass_df = pass_df.groupby(pass_col).all()
pass_df = pass_df.reset_index().loc[:, pass_col]

# rename columns

pass_df = pass_df.rename(columns={
    'passenger_email': 'passenger_email', 
    'passenger_first_name':'passenger_first_name', 
    'passenger_last_name':'passenger_last_name',
    'passenger_birth_date':'passenger_birth_date',
    'passenger_street':'passenger_street',
    'passenger_city':'passenger_city',
    'passenger_state':'passenger_state',
    'passenger_zip':'passenger_zip',
    })

# generate uuid for passenger id
pass_df.insert(0, 'passenger_id', uuid.uuid4())

# add start and end dates
pass_df['effective_start_date'] = "2006-06-06"
pass_df['effective_end_date'] = None

display(pass_df)



passenger_id                     passenger_email  \
0   277db099-ef2d-4511-a124-10d954f14cfd        annette.hawkins.43@yahoo.com   
1   277db099-ef2d-4511-a124-10d954f14cfd         autumn.morse.60@hotmail.com   
2   277db099-ef2d-4511-a124-10d954f14cfd            becky.clark.91@gmail.com   
3   277db099-ef2d-4511-a124-10d954f14cfd         belinda.cook.91@hotmail.com   
4   277db099-ef2d-4511-a124-10d954f14cfd          carl.wilson.80@hotmail.com   
5   277db099-ef2d-4511-a124-10d954f14cfd          cheryl.hughes.45@gmail.com   
6   277db099-ef2d-4511-a124-10d954f14cfd  christian.stevenson.93@hotmail.com   
7   277db099-ef2d-4511-a124-10d954f14cfd             corey.cook.83@gmail.com   
8   277db099-ef2d-4511-a124-10d954f14cfd   danielle.henderson.70@hotmail.com   
9   277db099-ef2d-4511-a124-10d954f14cfd           hannah.smith.66@gmail.com   
10  277db099-ef2d-4511-a124-10d954f14cfd          janice.zamora.00@gmail.com   
11  277db099-ef2d-4511-a124-10d954f14cfd        jennifer.garcia.50@gmail.com   
12  277db099-ef2d-4511-a124-10d954f14cfd        julie.carlson.77@hotmail.com   
13  277db099-ef2d-4511-a124-10d954f14cfd            kevin.moore.79@gmail.com   
14  277db099-ef2d-4511-a124-10d954f14cfd           laura.kent.98@hotmail.com   
15  277db099-ef2d-4511-a124-10d954f14cfd            laurie.york.84@gmail.com   
16  277db099-ef2d-4511-a124-10d954f14cfd          lisa.tucker.65@hotmail.com   
17  277db099-ef2d-4511-a124-10d954f14cfd          matthew.yates.75@yahoo.com   
18  277db099-ef2d-4511-a124-10d954f14cfd     megan.villanueva.45@hotmail.com   
19  277db099-ef2d-4511-a124-10d954f14cfd    nicholas.phillips.00@hotmail.com   
20  277db099-ef2d-4511-a124-10d954f14cfd           paul.baird.91@hotmail.com   
21  277db099-ef2d-4511-a124-10d954f14cfd         rachel.duffy.60@hotmail.com   
22  277db099-ef2d-4511-a124-10d954f14cfd         rebecca.wilson.73@yahoo.com   
23  277db099-ef2d-4511-a124-10d954f14cfd         robert.brown.69@hotmail.com   
24  277db099-ef2d-4511-a124-10d954f14cfd          ronald.cook.76@hotmail.com   
25  277db099-ef2d-4511-a124-10d954f14cfd             sarah.hall.44@gmail.com   
26  277db099-ef2d-4511-a124-10d954f14cfd         sarah.rivera.04@hotmail.com   
27  277db099-ef2d-4511-a124-10d954f14cfd            sean.murray.53@yahoo.com   
28  277db099-ef2d-4511-a124-10d954f14cfd          seth.thompson.68@yahoo.com   
29  277db099-ef2d-4511-a124-10d954f14cfd         shannon.rhodes.78@gmail.com   
30  277db099-ef2d-4511-a124-10d954f14cfd       timothy.santos.86@hotmail.com   
31  277db099-ef2d-4511-a124-10d954f14cfd           tony.hoffman.03@gmail.com   

   passenger_first_name passenger_last_name passenger_birth_date  \
0               Annette             Hawkins           1943-07-11   
1                Autumn               Morse           1960-01-18   
2                 Becky               Clark           1991-11-09   
3               Belinda                Cook           1991-01-26   
4                  Carl              Wilson           1980-04-24   
5                Cheryl              Hughes           1945-05-20   
6             Christian           Stevenson           1993-06-14   
7                 Corey                Cook           1983-06-14   
8              Danielle           Henderson           1970-08-11   
9                Hannah               Smith           1966-07-01   
10               Janice              Zamora           2000-06-14   
11             Jennifer              Garcia           1950-02-12   
12                Julie             Carlson           1977-04-05   
13                Kevin               Moore           1979-04-11   
14                Laura                Kent           1998-08-05   
15               Laurie                York           1984-04-28   
16                 Lisa              Tucker           1965-01-22   
17              Matthew               Yates           1975-03-31   
18                Megan          Villanueva           1945-08-14   
19             Nicholas    